## 그로스해킹 스터디_1회차 과제
- 학습 내용 : AARRR, 퍼널, 전환율, 이탈율
- 활용 데이터 : [kaggle e-commerce_website](https://www.kaggle.com/datasets/aerodinamicc/ecommerce-website-funnel-analysis?select=search_page_table.csv)
- 제작일 : 2025-10-22 21:39

<분석 흐름>
1. 데이터 불러오기
2. 데이터 병합 및 정제
3. 집계
4. 인사이트 도출

In [1]:
import pandas as pd
import os

# 1. 데이터 불러오기
user= pd.read_csv('01.ecommerce_website/user_table.csv')
homepage= pd.read_csv('01.ecommerce_website/home_page_table.csv')
search= pd.read_csv('01.ecommerce_website/search_page_table.csv')
payment= pd.read_csv('01.ecommerce_website/payment_page_table.csv')
payment_confirm= pd.read_csv('01.ecommerce_website/payment_confirmation_table.csv')

### 1. EDA

In [2]:
# 2. 월별 가입자수 집계
user['date']=pd.to_datetime(user['date'])
user['month']= user['date'].dt.strftime('%Y-%m')
user.groupby('month')['user_id'].nunique()

month
2015-01    22600
2015-02    22600
2015-03    22600
2015-04    22600
Name: user_id, dtype: int64

In [3]:
user['date'].value_counts()

date
2015-02-08    877
2015-02-07    846
2015-02-02    845
2015-02-15    835
2015-02-25    830
             ... 
2015-01-13    693
2015-03-27    693
2015-01-26    686
2015-03-19    684
2015-01-18    668
Name: count, Length: 120, dtype: int64

In [4]:
30200/90400

0.334070796460177

In [5]:
user['user_id'].nunique()

90400

### 2. 데이터 병합

In [6]:
step1 = pd.merge(user,homepage, on='user_id', how='left')
step2 = step1.merge(search, on='user_id',how='left',suffixes=('_home','_search'))
step3=step2.merge(payment, on='user_id',how='left').merge(payment_confirm, on='user_id',how='left',suffixes=('_payment','_confirm'))

In [7]:
step3

,user_id,date,device,sex,month,page_home,page_search,page_payment,page_confirm
0,450007,2015-02-28,Desktop,Female,2015-02,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,2015-01,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,2015-04,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,2015-02,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,2015-01,home_page,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
90395,307667,2015-03-30,Desktop,Female,2015-03,home_page,NaN,NaN,NaN
90396,642989,2015-02-08,Desktop,Female,2015-02,home_page,search_page,NaN,NaN
90397,659645,2015-04-13,Desktop,Male,2015-04,home_page,search_page,NaN,NaN
90398,359779,2015-03-23,Desktop,Male,2015-03,home_page,NaN,NaN,NaN


### 4. 데이터 집계 (전환율, 이탈율)

In [8]:
step_counts = step3.groupby('page_home')['user_id'].nunique()
step_counts[0]

C:\Users\damla\AppData\Local\Temp\ipykernel_18160\791337342.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  step_counts[0]


np.int64(90400)

In [9]:
step3.notnull().sum()

user_id         90400
date            90400
device          90400
sex             90400
month           90400
page_home       90400
page_search     45200
page_payment     6030
page_confirm      452
dtype: int64

In [10]:
home_num = step3['page_home'].notnull().sum()
search_num = step3['page_search'].notnull().sum()
payment_num = step3['page_payment'].notnull().sum()
confirm_num = step3['page_confirm'].notnull().sum()

In [11]:
data = {
    '가입' : [home_num],
    '검색' : [search_num],
    '주문' : [payment_num],
    '결제완료' : [confirm_num]
}

user_num_by_step = pd.DataFrame(data, index=['유저수 (unique)'])
user_num_by_step

,가입,검색,주문,결제완료
유저수 (unique),90400,45200,6030,452


In [12]:
# 전환율 / 이탈율 계산
conversion_rate = [
    round(search_num / home_num * 100, 2),
    round(payment_num / search_num * 100, 2),
    round(confirm_num / payment_num * 100, 2)
]

churn_rate = [round(100 - c, 2) for c in conversion_rate]

# 전환율/이탈율 행 추가
user_num_by_step.loc['전환율 (%)'] = ['-', *conversion_rate]
user_num_by_step.loc['이탈율 (%)'] = ['-', *churn_rate]

In [13]:
user_num_by_step

,가입,검색,주문,결제완료
유저수 (unique),90400,45200.0,6030.00,452.0
전환율 (%),-,50.0,13.34,7.5
이탈율 (%),-,50.0,86.66,92.5


In [14]:
display(
    user_num_by_step.style
    .set_caption("과제1) 각 페이지별 전환율")
    .format(
        lambda x: "-" if pd.isna(x) or isinstance(x, str) else f"{x:.1f}"
    )
    .background_gradient(subset=['검색', '주문', '결제완료'], cmap='Blues')
)


,가입,검색,주문,결제완료
유저수 (unique),90400.0,45200.0,6030.0,452.0
전환율 (%),-,50.0,13.3,7.5
이탈율 (%),-,50.0,86.7,92.5


### 장치별, 성별 전환율

In [17]:
def analysis_detail(column, feature):
    #1.단계별 값 집계
    home_num = step3[step3[column]==feature]['page_home'].notnull().sum()
    search_num = step3[step3[column]==feature]['page_search'].notnull().sum()
    payment_num = step3[step3[column]==feature]['page_payment'].notnull().sum()
    confirm_num = step3[step3[column]==feature]['page_confirm'].notnull().sum()

    #2.숫자로 저장
    data = {
    '가입' : [home_num],
    '검색' : [search_num],
    '주문' : [payment_num],
    '결제완료' : [confirm_num]
    }

    user_num_by_step = pd.DataFrame(data, index=['유저수 (unique)'])
    
    #3. 전환율, 이탈율 계산

    conversion_rate = [
        round(search_num / home_num * 100, 2),
        round(payment_num / search_num * 100, 2),
        round(confirm_num / payment_num * 100, 2)
    ]

    churn_rate = [round(100 - c, 2) for c in conversion_rate]

    # 전환율/이탈율 행 추가
    user_num_by_step.loc['전환율 (%)'] = ['-', *conversion_rate]
    user_num_by_step.loc['이탈율 (%)'] = ['-', *churn_rate]
    
    return display(
        user_num_by_step.style
        .set_caption(f"과제1) 각 페이지별 전환율_{column},{feature}")
        .format(
            lambda x: "-" if pd.isna(x) or isinstance(x, str) else f"{x:.1f}"
        )
        .background_gradient(subset=['검색', '주문', '결제완료'], cmap='Blues')
    )


In [18]:
analysis_detail('device', 'Desktop')

,가입,검색,주문,결제완료
유저수 (unique),60200.0,30100.0,3010.0,150.0
전환율 (%),-,50.0,10.0,5.0
이탈율 (%),-,50.0,90.0,95.0


In [19]:
analysis_detail('device', 'Mobile')

,가입,검색,주문,결제완료
유저수 (unique),30200.0,15100.0,3020.0,302.0
전환율 (%),-,50.0,20.0,10.0
이탈율 (%),-,50.0,80.0,90.0


In [20]:
analysis_detail('sex', 'Female')

,가입,검색,주문,결제완료
유저수 (unique),45075.0,22676.0,3100.0,241.0
전환율 (%),-,50.3,13.7,7.8
이탈율 (%),-,49.7,86.3,92.2


In [21]:
analysis_detail('sex', 'Male')

,가입,검색,주문,결제완료
유저수 (unique),45325.0,22524.0,2930.0,211.0
전환율 (%),-,49.7,13.0,7.2
이탈율 (%),-,50.3,87.0,92.8


In [22]:
# 전체 전환율
452/90400

0.005

In [23]:
### 기초 과제용 데이터 저장
step3.to_csv("GH_1회차_과제용_ecommerce_website.csv",index=False)